# Australia Meteorological Data Viewer

This notebook loads and visualizes the mock meteorological data (temperature and precipitation) stored in zarr format.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

# Set plot style
plt.style.use('default')
%matplotlib inline

## Load the Dataset

In [ ]:
# Load the observation dataset (contains tmax and precip variables)
ds = xr.open_zarr('../data/australia_obs.zarr')
print(ds)

## Dataset Information

In [ ]:
# Display metadata
print("\nDataset Attributes:")
for key, value in ds.attrs.items():
    print(f"  {key}: {value}")

print("\nAvailable Variables:")
for var in ds.data_vars:
    print(f"\n{var} - {ds[var].attrs.get('long_name', 'No description')}")
    for key, value in ds[var].attrs.items():
        if key != 'long_name':
            print(f"  {key}: {value}")

## Visualize Data Map

Select a variable and day to visualize the data distribution across Australia.

In [ ]:
# ========================================
# SELECT VARIABLE AND DAY
# ========================================
# Change these values to view different variables and days

selected_variable = 'tmax'  # Options: 'tmax' or 'precip'
selected_day = 6           # Options: 0-9 (Day 0 = first day, Day 9 = last day)

# Variable display settings
variable_config = {
    'tmax': {
        'cmap': 'RdYlBu_r',  # Red-Yellow-Blue reversed (hot to cold)
        'vmin': 15,
        'vmax': 45,
        'unit': '°C',
        'title': 'Daily Maximum Temperature'
    },
    'precip': {
        'cmap': 'Blues',  # Blue colormap for precipitation
        'vmin': 0,
        'vmax': 30,
        'unit': 'mm',
        'title': 'Daily Precipitation'
    }
}

print(f"Selected: {variable_config[selected_variable]['title']}")
print(f"Day: {selected_day} ({str(ds.time.values[selected_day])[:10]})")

In [ ]:
# Load Australia boundary
aus_boundary = gpd.read_file('../data/australia_boundary.geojson')

# Get the configuration for the selected variable
config = variable_config[selected_variable]

# Get data for selected variable and day
data_day = ds[selected_variable].isel(time=selected_day)
date_str = str(ds.time.values[selected_day])[:10]

# Create a figure
fig, ax = plt.subplots(figsize=(14, 10))

# Plot the data
im = ax.pcolormesh(
    ds.lon, 
    ds.lat, 
    data_day,
    cmap=config['cmap'],
    shading='auto',
    vmin=config['vmin'],
    vmax=config['vmax']
)

# Plot Australia boundary on top
aus_boundary.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5, label='Australia')

# Add colorbar
cbar = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02, shrink=0.8)
cbar.set_label(f'{config["title"]} ({config["unit"]})', fontsize=12)

# Set labels and title
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)
ax.set_title(f'Australia - {config["title"]}\nDay {selected_day}: {date_str}', 
             fontsize=14, fontweight='bold')

# Set aspect ratio to make it look more geographic
ax.set_aspect('equal')

# Add grid
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

# Add legend
ax.legend(loc='upper right')

# Add text showing max and mean values for this day
max_val = float(data_day.max())
mean_val = float(data_day.mean())
ax.text(0.02, 0.98, f'Max: {max_val:.1f}{config["unit"]}\nMean: {mean_val:.1f}{config["unit"]}', 
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

## Time Series at Specific Location

Example: Get time series for the selected variable at a specific coordinate

In [ ]:
# Example coordinates (roughly Sydney area)
example_lat = -33.9
example_lon = 151.2

# Get the configuration for the selected variable
config = variable_config[selected_variable]

# Get data time series at this location across all 10 days
point_data = ds[selected_variable].sel(lat=example_lat, lon=example_lon, method='nearest')

# Get actual coordinates
actual_lat = float(point_data.lat)
actual_lon = float(point_data.lon)

# Extract values for each day
values = [float(point_data.isel(time=i)) for i in range(len(ds.time))]
dates = [str(t)[:10] for t in ds.time.values]
days = range(len(ds.time))

# Create time series plot
fig, ax = plt.subplots(figsize=(12, 6))

# Set color based on variable
plot_color = 'darkred' if selected_variable == 'tmax' else 'darkblue'
marker_color = 'red' if selected_variable == 'tmax' else 'blue'

ax.plot(days, values, 'o-', color=plot_color, linewidth=2.5, markersize=10, 
        markerfacecolor=marker_color, markeredgecolor=plot_color)

# Highlight the selected day
ax.plot(selected_day, values[selected_day], 'o', markersize=15, markerfacecolor='yellow', 
        markeredgecolor='black', markeredgewidth=2, zorder=5, label=f'Selected Day ({selected_day})')

ax.set_xlabel('Day', fontsize=12)
ax.set_ylabel(f'{config["title"]} ({config["unit"]})', fontsize=12)
ax.set_title(f'10-Day {config["title"]} at Location ({actual_lat:.2f}°, {actual_lon:.2f}°)\nNear Sydney', 
             fontsize=14, fontweight='bold')
ax.set_xticks(days)
ax.set_xticklabels([f'Day {i}\n{dates[i][5:]}' for i in days], fontsize=9)

# Set y-axis limits to accommodate labels
val_range = max(values) - min(values)
ax.set_ylim(min(values) - val_range * 0.1, max(values) + val_range * 0.15)

ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

# Add value labels on each point
for i, val in enumerate(values):
    ax.text(i, val + val_range * 0.02, f'{val:.1f}{config["unit"]}', 
            ha='center', fontsize=9, color=plot_color)

plt.tight_layout()
plt.show()

print(f"\nLocation: ({actual_lat:.2f}°, {actual_lon:.2f}°)")
print(f"{config['title']} range: {min(values):.1f}{config['unit']} to {max(values):.1f}{config['unit']}")
print(f"{config['title']} change: {values[-1] - values[0]:.1f}{config['unit']} (Day 0 to Day 9)")